# Please use the space below to provide your comments about what you believe is our company's greatest opportunity. Your comments will appear exactly as written, and will not be edited in any way.

In [1]:
import nltk
import numpy as np
import pandas as pd

In [2]:
question = 'OC0002'

comment = pd.read_csv('..\data\ManuLife_OC0002.csv',index_col = 0,names = ['EmployeeID','Comment'])

In [3]:
comment = comment.fillna('No Comment')

In [4]:
print(comment.head())
print(comment.shape)

                                                      Comment
EmployeeID                                                   
580607                                         Cultural shift
580609      I can only think of 2 opportunities at the mom...
580611      Most people will leave the company because oth...
580612               to be able to get a larger target market
580615                Continue to make employees feel valued.
(9033, 1)


# Three steps
## 1 - tokenize
## 2 - remove stop words
## 3 - stem

In [5]:
#make everything lower case
comment['Comment'] = comment['Comment'].str.lower()

In [6]:
# tokenize main comment

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
comment['Comment_Tokenized'] = None

for i in range(0,comment.shape[0]):
    tkn = tokenizer.tokenize(comment.iloc[i,0])
    comment['Comment_Tokenized'].iloc[i] = tkn

In [7]:
#remove stop words

import string
from nltk.corpus import stopwords
stop = stopwords.words('english')
                       
#remove punctuation from stop words
stop = [''.join(c for c in s if c not in string.punctuation) for s in stop]

#add in key words from the question
question_kw = ['company'
               ,'greatest','opportunity','believe']
stop.extend(question_kw)

comment['Comment_NoStop'] = None

for i in range(0,comment.shape[0]):
    text = comment['Comment_Tokenized'].iloc[i]
    text = [word for word in text if word not in stop]
    comment['Comment_NoStop'].iloc[i] = text

In [8]:
#stem the words

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmer = PorterStemmer()

comment['Comment_stemmed'] = None

for i in range(0,comment.shape[0]):
    t = comment['Comment_NoStop'].iloc[i]
    
    l = []

    for word in t:
        a = stemmer.stem(word)
        l.append(a)
        
    comment['Comment_stemmed'].iloc[i] = l

In [9]:
bigram_cols = ['EmployeeID','Bigram']
bigram_df = pd.DataFrame(columns = bigram_cols)

In [10]:
import time

t0 = time.time()

for i in range(0,comment.shape[0]):
    test = list(nltk.bigrams(comment['Comment_stemmed'].iloc[i]))
    eid = comment.index.values[i]
    for t in test:
        d_dict ={'EmployeeID' : [eid], 'Bigram' : [t]}
        df_temp = pd.DataFrame(data = d_dict)
        bigram_df = bigram_df.append(df_temp)
        
t1 = time.time()
print('{:.2} minutes to run'.format((t1-t0)/60))

5.1 minutes to run


In [12]:
bigram_df = bigram_df.reset_index(drop = True).copy()
bigram_df['Bigram1'],bigram_df['Bigram2'] = zip(*bigram_df['Bigram'])
bigram_df['Bigram_Clean'] = bigram_df['Bigram1'] + ' ' + bigram_df['Bigram2']

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
comment['Sentiment'] = None
sid = SentimentIntensityAnalyzer()

for i in range(0,comment.shape[0]):
    t = comment['Comment'].iloc[i]
    ss = sid.polarity_scores(t)
    comment['Sentiment'].iloc[i] = ss['compound']

In [ ]:
comment.to_csv('~/Desktop/comments.csv')

In [24]:
bigram_df.to_csv('~/Desktop/bigrams.csv')

In [ ]:
#how do we do topic extraction